<a href="https://colab.research.google.com/github/bimal-bp/Hybrid_search_rag_project/blob/main/Hybrid_Search_using_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Installation Requirements
!pip install llama-index llama-index-llms-gemini llama-index-vector-stores-qdrant fastembed
!pip install llama-index-embeddings-fastembed

In [3]:
# Step 2: Define LLM and Embedding Model
import os
from getpass import getpass
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.fastembed import FastEmbedEmbedding

GOOGLE_API_KEY = getpass("Enter your Gemini API:")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

llm = Gemini() # gemini 1.5 flash
embed_model = FastEmbedEmbedding()

Enter your Gemini API:··········


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
# Now let’s test if the API is currently defined by running that LLM on a sample user query.
llm_response = llm.complete("who is the captaion of indian odi team ").text
print(llm_response)

The current captain of the Indian ODI team is **Rohit Sharma**. 



In LlamaIndex, OpenAI is the default LLM and Embedding model, to override that we need to define Settings from LlamaIndex Core. Here we need to override both LLM and Embed model.

In [5]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [11]:
# Step 3: Loading Your Data

from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/").load_data()

In [12]:
# Step 4: Setting Up Qdrant with Hybrid Search

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

client = qdrant_client.QdrantClient(
    location=":memory:",
)

vector_store = QdrantVectorStore(
    collection_name = "paper",
    client=client,
    enable_hybrid=True, # Hybrid Search will take place
    batch_size=20,
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/532M [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
vector_store

QdrantVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=False, collection_name='paper', url=None, api_key=None, batch_size=20, parallel=1, max_retries=3, client_kwargs={}, enable_hybrid=True, index_doc_id=True, fastembed_sparse_model=None)

In [15]:
# Step 5: Indexing your document
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)


In [17]:
# Step 6: Querying the Index Query Engine
query_engine = index.as_query_engine(
    vector_store_query_mode="hybrid"
)

response1 = query_engine.query("what is the meaning of self attention?")
print(response1)
response2 = query_engine.query("give the defination of python")
print(response2)

Self-attention allows each position in the encoder or decoder to attend to all positions in the previous layer. 

The provided text does not contain information about Python. 



In [18]:
# Step 7: Define Memory
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=3000)

In [19]:
# Step 8: Creating a Chat Engine with Memory
chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
        "You are an AI assistant who answers the user questions"
    ),
)

In [20]:
# Step 9: Testing Memory
from IPython.display import Markdown, display

check1 = chat_engine.chat("give the abstract within 2 sentence")

check2 = chat_engine.chat("continue the abstract, add one more sentence to the previous two sentence")

check3 = chat_engine.chat("make the above abstract into a poem")

In [21]:
print(check1)

This paper introduces the Transformer, a novel neural network architecture for sequence transduction tasks, such as machine translation.  The Transformer relies entirely on attention mechanisms, eliminating the need for recurrent or convolutional layers, and achieves state-of-the-art results on English-to-German and English-to-French translation tasks. 



In [23]:
print(check2)

This paper introduces the Transformer, a novel neural network architecture for sequence transduction tasks, such as machine translation.  The Transformer relies entirely on attention mechanisms, eliminating the need for recurrent or convolutional layers, and achieves state-of-the-art results on English-to-German and English-to-French translation tasks.  The Transformer is parallelizable and requires significantly less time to train than recurrent models, making it a promising new approach for sequence transduction. 



In [24]:
print(check3)

A new machine, the Transformer named,
For tasks of sequence, it's acclaimed.
No need for loops, no convolutions deep,
Attention's power, secrets it will keep.

On English tongues, to German, French it goes,
State-of-the-art, its prowess it bestows.
Faster than the rest, it learns with grace,
A parallel world, it sets the pace. 



**Conclusion**

We explored how integrating memory and hybrid search into Retrieval Augmented Generation (RAG) systems significantly enhances their capabilities. By using LlamaIndex with Qdrant as the vector store and Google’s Gemini as the Large Language Model, we demonstrated how hybrid search can combine the strengths of vector and keyword-based retrieval to deliver more precise results. The addition of memory further improved contextual understanding, allowing the chatbot to provide coherent responses across multiple interactions. Together, these features create a more intelligent and context-aware system, making RAG pipelines more effective for complex AI applications